<a href="https://colab.research.google.com/github/deepanrajm/deep_learning/blob/master/Transfer-Learning/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14 tensorflow-gpu==1.14 keras==2.1.2 h5py==2.10.0

In [13]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19, preprocess_input

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [ ]:
!git clone https://github.com/deepanrajm/deep_learning.git

In [ ]:
!ls

In [14]:
# Load the training data set by looping over every image file
for image_file in Path("deep_learning/Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(100, 100))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [ ]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
base_model = VGG19(weights="imagenet")
feature_extractor = VGG19(include_top=False, input_shape=(100, 100, 3))

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


In [16]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [18]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [ ]:
# Train the model
model.fit(
    x_train,
    y_train,
    validation_split=0.05,
    epochs=10,
    shuffle=True,
    verbose=2
)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_feature_classifier_model.h5")

In [20]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix

In [23]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("deep_learning/Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(100, 100))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

In [ ]:
# Load our trained classifier model
model = load_model("bird_feature_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")